# <CENTER> EXAMEN FINAL DE SISTEMAS EXPERTOS</CENTER>


### Enunciado:

Se desea generar un sistema de recomendación de películas, por tal motivo se va a utiliza una base de datos orientada a grafos y un control de lógica difusa para clasificar el riesgo financiero, el mismo que sera ingresado como atributo del cliente en el sistema re comendador, para lograr esto se describe los pasos a seguir:
1)	Evaluar el riesgo financiero de sus clientes que requieren la recomendación de películas. Para evaluar el riesgo financiero se toma en cuenta la edad del asegurado y su porcentaje de manejo durante el año. Para ello se tiene las siguientes reglas y la función de pertinencia. El proceso seguir se describe en  el siguiente link: https://medium.com/@javierdiazarca/l%C3%B3gica-difusa-ejercicios-propuestos-b99603ef1bc0.

2)	Generar números aleatorios para la edad y el porcentaje de manejo con el objetivo de generar al menos 100 personas y ademas incluir el listado de películas vistas y el valor del rating de cada película. Al menos 50 películas y un total de nodos de al menos 350 nodos.

3)	Con estos datos aplicar el algoritmo de KNN y Similitud de Coseno para la recomendación de películas, seguir el siguiente tutorial: https://neo4j.com/graphgist/movie-recommendations-with-k-nearest-neighbors-and-cosine-similarity.

4)	Finalmente realizar alguna interfaz para poder acceder a la recomendación e ingreso de datos y resultados de los procesos.


Generar el Informe en PDF y subir los scripts al repositorio Git para su evaluación.


Criterios de Evaluación

- Sistema lógico difuso: 30%
- Neo4J Knn: 30%
- Informe y resultados: 20%
- GUI. programación y pruebas: 20%




# DESARROLLO DEL RIESGO FINANCIERO

In [ ]:
import numpy as np
import skfuzzy as fuzz
import random
from skfuzzy import control as ctrl

# New Antecedent/Consequent objects hold universe variables and membership
# functions
edad = ctrl.Antecedent(np.arange(18, 70, 1), 'edad')
manejo = ctrl.Antecedent(np.arange(0, 100, 1), 'manejo')
RF = ctrl.Consequent(np.arange(0, 100, 1), 'RF')

# Auto-membership function population is possible with .automf(3, 5, or 7)
edad.automf(3)
manejo.automf(3)

# Pythonic API
RF['medium'] = fuzz.trimf(RF.universe, [10, 30, 45])
RF['high'] = fuzz.trimf(RF.universe, [40, 60, 100])

edad.view()
manejo['average'].view()
RF.view()

In [ ]:
#reglas
rule1 = ctrl.Rule(edad['poor'] | manejo['average'], RF['high'])
rule2 = ctrl.Rule(edad['average'] | manejo['average'], RF['medium'])

rule1.view()
rule2.view()
print(rule1)
print(rule2)


In [ ]:
#Ahora que tenemos nuestras reglas definidas, podemos simplemente crear un sistema de control a través de:
tipping_ctrl = ctrl.ControlSystem([rule1, rule2])
tipping = ctrl.ControlSystemSimulation(tipping_ctrl)

In [ ]:
# Ahora podemos simular nuestro sistema de control simplemente especificando las entradas y llamando al computemétodo.

Iedad = int(random.randrange(18,70))
Imanejo = int(random.randrange(0,100))
print(Iedad)
print(Imanejo)

#tipiamos los datos randomicos
tipping.input['edad'] = Iedad
tipping.input['manejo'] = Imanejo
tipping.compute()

In [ ]:
#Una vez calculado, podemos ver el resultado así como visualizarlo.

rFinanciero=tipping.output['RF']

if tipping.output['RF'] > 50:
    print("su riesgo finaciero es Alto, Peligro:  ")
    print(rFinanciero)
else:
    print("su riesgo finaciero es medio, tenga precaucion: ")
    print(rFinanciero)
   
RF.view(sim=tipping)

In [ ]:
persona=1
neo4j = Neo4jService('bolt://localhost:7687', 'neo4j', 'final')
with neo4j._driver.session() as session:
        for i in (np.arange(0,100, 1)):
            name="persona"+str(persona)
            session.write_transaction(neo4j.cnodo, nombre, r)
            persona=persona+1
        

In [18]:
from neo4j import GraphDatabase

class Neo4jService(object):

    def __init__(self, uri, user, password):
        self._driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self._driver.close()
    
    def recomendacion(self,tx):
        result = tx.run("MATCH(n) return(n) ")
        print(result)
        
    def cnodo(self, tx, nombre,pre):
        tx.run("MERGE (persona:Person {name:$nombre})"    
        "SET persona.embedding = [$pre]",nombre=nombre
        ,pre=rFinanciero)

In [ ]:
from tkinter import *
from tkinter import ttk
from tkinter import messagebox
import tkinter as tk

vent = Tk()
        
def listar():
    neo4j = Neo4jService('bolt://localhost:7687', 'neo4j', 'final')
    with neo4j._driver.session() as session:
        session.read_transaction(neo4j.recomendacion)
        


def crear():
    neo4j = Neo4jService('bolt://localhost:7687', 'neo4j', 'final')
    with neo4j._driver.session() as session:
        session.write_transaction(neo4j.cnodo , nombre.get(),float(rFinaciero.get()))
        

    
vent.geometry('400x400')
vent.title('Examen')

Label(vent, text="Nuevo Usuario:").place(x=55, y=15)
nombre = ttk.Entry(vent)
nombre.place(x = 150, y=15)
ttk.Button(vent, text='Crear Nuevo Usuario', command=crear).place(x=150, y=55)


Label(vent, text="Prefieres jugar en un parque con arboles?").place(x=55, y=100)
pr = Spinbox(vent, from_=0, to=1, width=5, increment=1)
pr.place(x=320, y=100)
ttk.Button(vent, text='Recomendar', command=listar).place(x=200, y=130)

Label(vent, text="Se recomienda visitar estos lugares.").place(x=80, y=160)  
    
resultado = Text(vent)
resultado.place(x = 90, y=200, width=300, height=100)

    
vent.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "/Users/zhimi/opt/anaconda3/lib/python3.8/tkinter/__init__.py", line 1883, in __call__
    return self.func(*args)
  File "<ipython-input-19-0891009174ce>", line 18, in crear
    session.write_transaction(neo4j.cnodo , nombre.get(),float(rFinaciero.get()))
NameError: name 'rFinaciero' is not defined
